In [1]:
import numpy as np
import math
from numpy import linalg as la

**Euler2A**

In [2]:
def euler_2_a(fi, teta, psi):
    
    Rx_fi = np.array([[1, 0, 0],
                    [0, math.cos(fi), -math.sin(fi)],
                    [0, math.sin(fi), math.cos(fi)]])
    
    Ry_teta = np.array([[math.cos(teta), 0, math.sin(teta)],
                      [0, 1, 0],
                      [-math.sin(teta), 0, math.cos(teta)]])
    
    Rz_psi = np.array([[math.cos(psi), -math.sin(psi), 0],
                     [math.sin(psi), math.cos(psi), 0],
                     [0, 0, 1]])
    
    A = np.matmul(Ry_teta, Rx_fi)
    A = np.matmul(Rz_psi, A)
    A = np.round(A, 6)
    
    return A

In [3]:
uglovi = [-math.atan(1/4), -math.asin(8/9), math.atan(4)]
A = euler_2_a(uglovi[0], uglovi[1], uglovi[2])
A = np.round(A, 6)
print(A)

[[ 0.111111 -0.888889 -0.444444]
 [ 0.444444  0.444444 -0.777778]
 [ 0.888889 -0.111111  0.444444]]


**AxisAngle**

In [4]:
def axis_angle(A):
    
    if round(la.det(A), 4) != 1:
        print("Determinanta matrice A mora biti 1")
        print(round(la.det(A), 5))
        return

    if np.any(np.round(A.dot(A.T), 5) != np.eye(3)):
        print("Matrica A mora biti ortogonalna")
        return
    
# odredjivanje sopstvenog vektora
    lambdas, vector = la.eig(A)
    for i in range(len(lambdas)):
        if np.round(lambdas[i], 6) == 1.0:
            p = np.real(vector[:, i])

    p1, p2, p3 = p[0], p[1], p[2]
    
    if p1 == 0 and p2 == 0 and p3 == 0:
        print("Ne sme biti nula vektor")
        return
        
# u je izabran prozvoljni jedinicni vektor koji je normalan na p 
    u = np.cross(p, np.array([1, 1, 1]))
    u = u/math.sqrt(u[0]**2+u[1]**2+u[2]**2)

    up = A.dot(u)

    fi = np.round(math.acos(np.dot(u, up)), 6)
    if np.round(np.dot(np.cross(u, up), p), 6) < 0:
        p = (-1)*p

    
    return [p, fi]

In [5]:
[p, fi] = axis_angle(A)
print(p, fi)
print([p, f'{np.round(fi/math.pi, 5)}*PI'])

[ 0.33333363 -0.66666654  0.66666665] 1.570797
[array([ 0.33333363, -0.66666654,  0.66666665]), '0.5*PI']


**Rodrigezova formula**

In [6]:
def rodrigez(p, fi):

    norma = la.norm(p)
    p /= norma

    Px = np.array([[0, -p[2], p[1]],
                   [p[2], 0, -p[0]],
                   [-p[1], p[0], 0]])

    E = np.eye(3)
    p = np.reshape(p, (3, 1))
    Rp = p.dot(p.T) + math.cos(fi)*(E - p.dot(p.T)) + math.sin(fi)*Px
    Rp = np.round(Rp, 6)

    return Rp

In [7]:
R = rodrigez(p, fi)
print(R)
print()
print(A)

[[ 0.111111 -0.888889 -0.444444]
 [ 0.444444  0.444444 -0.777778]
 [ 0.888889 -0.111111  0.444444]]

[[ 0.111111 -0.888889 -0.444444]
 [ 0.444444  0.444444 -0.777778]
 [ 0.888889 -0.111111  0.444444]]


**A2Euler**

In [8]:
def a_2_euler(A):
    
    if round(la.det(A)) != 1:
        print("Determinanta matrice A mora biti 1")
        return

    if np.any(np.round(A.dot(A.T), 5) != np.eye(3)):
        print("Matrica A mora biti ortogonalna")
        return
    
    fi, teta, psi = 0, 0, 0
    if A[2, 0] < 1:
        if A[2, 0] > -1:
            psi = math.atan2(A[1, 0], A[0, 0])
            teta = math.asin(-A[2, 0])
            fi = math.atan2(A[2, 1], A[2, 2])
        else:
            psi = math.atan2((-1)*A[0, 1], A[1, 1])
            teta = math.pi/2.0
            fi = 0.0
    else:
        psi = math.atan2((-1)*A[0, 1], A[1, 1])
        teta = - math.pi/2.0
        fi = 0

    return([fi, teta, psi])

In [9]:
print(a_2_euler(A))
print(uglovi)

[-0.24497866312686414, -1.0949143196701623, 1.3258176636680326]
[-0.24497866312686414, -1.09491407713448, 1.3258176636680326]


**AxisAngle2Q**

In [10]:
def axis_angle_2_q(p, fi):
    
    norma = la.norm(p)
    
    if(norma == 0):
        print('Ne sme biti nula vektor')
        return
        
    p /= norma
        
    [x, y, z] = math.sin(fi/2.0) * p
    w = round(math.cos(fi/2.0), 6)

    return [x, y, z, w]

In [11]:
q = axis_angle_2_q(p, fi)
print(q)

[0.23570254924666617, -0.47140458780496247, 0.47140466637233047, 0.707107]


**Q2AxisAngle**

In [12]:
def q_2_axis_angle(q):
    
    if q[0] == 0 and q[1] == 0 and q[2] == 0:
        print("Imaginarni deo ne sme biti 0")
        return
    
    norma = la.norm(q)
    q /= norma

    fi = 2*math.acos(q[3])
    fi = round(fi, 6)
    
    if abs(q[3]) == 1:
        p = [1, 0, 0]
    else:
        norma = la.norm(np.array([q[0], q[1], q[2]]))
        p = np.array([q[0], q[1], q[2]])
        if norma != 0:
            p = p / norma

    return [p, fi]

In [13]:
print(q_2_axis_angle(q))
print([p, fi])

[array([ 0.33333363, -0.66666654,  0.66666665]), 1.570796]
[array([ 0.33333363, -0.66666654,  0.66666665]), 1.570797]


**Testiranje**

In [14]:
# zadati pocetni ojlerovi uglovi
fi = -math.atan(1/math.sqrt(3))
teta = -math.asin(1/math.sqrt(2))
psi = math.atan(math.sqrt(3))

print([fi, teta, psi])

[-0.5235987755982989, -0.7853981633974482, 1.0471975511965976]


In [15]:
# racunanje matrice preko ojlerovih uglova
A = euler_2_a(fi, teta, psi)
print(A)

[[ 0.353553 -0.573223 -0.739199]
 [ 0.612372  0.739199 -0.28033 ]
 [ 0.707107 -0.353553  0.612372]]


In [16]:
# racunanje ose rotacije i ugla preko matrice
[p, alfa] = axis_angle(A)
print([p, alfa])

[array([-0.03912368, -0.77277425,  0.63347399]), 1.210489]


In [17]:
# racunjanje matrice preko ose i ugla rotacije
# primetimo da se dobije ista matrica kao kada se racuna preko ojlerovih uglova
print(rodrigez(p, alfa))
print()
print(A)

[[ 0.353553 -0.573223 -0.739199]
 [ 0.612372  0.739199 -0.28033 ]
 [ 0.707107 -0.353553  0.612372]]

[[ 0.353553 -0.573223 -0.739199]
 [ 0.612372  0.739199 -0.28033 ]
 [ 0.707107 -0.353553  0.612372]]


In [18]:
# racunjanje ojlerovih uglova preko matrice
# primetimo da se dobiju pocetni uglovi
print(a_2_euler(A))
print([fi, teta, psi])

[-0.523598605304503, -0.7853984728464484, 1.0471977214903936]
[-0.5235987755982989, -0.7853981633974482, 1.0471975511965976]


In [19]:
# racunanje kvaterniona preko ose i ugla rotacije
q = axis_angle_2_q(p, alfa)
print(q)

[-0.022259933156387655, -0.4396800778565036, 0.3604233664860778, 0.822363]


In [20]:
# racunjanje ose i ugla rotacije preko kvaterniona
# primetimo da se dobije isti rezultat kao kada se racuna preko matrice
print(q_2_axis_angle(q))
print([p, alfa])

[array([-0.03912368, -0.77277425,  0.63347399]), 1.210489]
[array([-0.03912368, -0.77277425,  0.63347399]), 1.210489]
